In [3]:
# Core Libraries for LOF Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [2]:
# Data Generation Functions
def create_anomaly_datasets(random_state=42):
    """
    Create various datasets with known anomalies for LOF demonstration
    
    Returns:
    --------
    datasets : dict
        Dictionary containing different anomaly detection scenarios
    """
    np.random.seed(random_state)
    
    datasets = {}
    
    # Dataset 1: Two clusters with scattered outliers
    X1, _ = make_blobs(n_samples=200, centers=2, cluster_std=1.0, random_state=random_state)
    # Add outliers
    outliers1 = np.random.uniform(low=-8, high=8, size=(20, 2))
    X1_with_outliers = np.vstack([X1, outliers1])
    y1 = np.concatenate([np.zeros(200), np.ones(20)])  # 0=normal, 1=outlier
    datasets['two_clusters'] = (X1_with_outliers, y1)
    
    # Dataset 2: Single cluster with outliers
    X2, _ = make_blobs(n_samples=150, centers=1, cluster_std=1.5, random_state=random_state)
    # Add outliers at different distances
    outliers2a = np.random.normal(loc=6, scale=0.5, size=(10, 2))  # Far outliers
    outliers2b = np.random.normal(loc=-6, scale=0.5, size=(10, 2))  # Far outliers
    outliers2c = np.random.normal(loc=0, scale=4, size=(10, 2))  # Medium outliers
    X2_with_outliers = np.vstack([X2, outliers2a, outliers2b, outliers2c])
    y2 = np.concatenate([np.zeros(150), np.ones(30)])
    datasets['single_cluster'] = (X2_with_outliers, y2)
    
    # Dataset 3: Multiple clusters with varying densities
    X3a, _ = make_blobs(n_samples=80, centers=1, cluster_std=0.8, center_box=(0, 2), random_state=random_state)
    X3b, _ = make_blobs(n_samples=50, centers=1, cluster_std=1.5, center_box=(4, 6), random_state=random_state+1)
    X3c, _ = make_blobs(n_samples=30, centers=1, cluster_std=0.5, center_box=(-3, -2), random_state=random_state+2)
    # Add outliers
    outliers3 = np.array([[8, 8], [-6, 6], [2, -4], [-2, 8], [6, -2]])
    X3_combined = np.vstack([X3a, X3b, X3c, outliers3])
    y3 = np.concatenate([np.zeros(160), np.ones(5)])
    datasets['varying_density'] = (X3_combined, y3)
    
    # Dataset 4: Linear pattern with outliers
    t = np.linspace(0, 4*np.pi, 100)
    X4 = np.column_stack([t, np.sin(t) + 0.2*np.random.randn(100)])
    # Add outliers perpendicular to the pattern
    outliers4 = np.array([[2, 3], [4, -3], [8, 2], [10, -2], [6, 4]])
    X4_with_outliers = np.vstack([X4, outliers4])
    y4 = np.concatenate([np.zeros(100), np.ones(5)])
    datasets['linear_pattern'] = (X4_with_outliers, y4)
    
    return datasets

# LOF Analysis Functions
def analyze_lof_sensitivity(X, y, k_range=range(5, 51, 5), contamination_range=None):
    """
    Analyze LOF sensitivity to parameters
    
    Parameters:
    -----------
    X : array-like
        Input features
    y : array-like
        True labels (0=normal, 1=outlier)
    k_range : iterable
        Range of k values to test
    contamination_range : iterable
        Range of contamination values to test
    
    Returns:
    --------
    results : dict
        Analysis results including optimal parameters
    """
    if contamination_range is None:
        contamination_range = [0.05, 0.1, 0.15, 0.2, 0.25]
    
    results = []
    
    print("Analyzing LOF parameter sensitivity...")
    print(f"Testing {len(k_range)} k values and {len(contamination_range)} contamination values...")
    
    for k in k_range:
        for contamination in contamination_range:
            try:
                # Fit LOF
                lof = LocalOutlierFactor(n_neighbors=k, contamination=contamination)
                y_pred = lof.fit_predict(X)
                lof_scores = -lof.negative_outlier_factor_
                
                # Convert predictions (-1, 1) to (1, 0)
                y_pred_binary = (y_pred == -1).astype(int)
                
                # Calculate metrics
                if len(np.unique(y)) > 1 and len(np.unique(y_pred_binary)) > 1:
                    auc_score = roc_auc_score(y, lof_scores)
                    precision = np.sum((y == 1) & (y_pred_binary == 1)) / np.sum(y_pred_binary == 1) if np.sum(y_pred_binary == 1) > 0 else 0
                    recall = np.sum((y == 1) & (y_pred_binary == 1)) / np.sum(y == 1) if np.sum(y == 1) > 0 else 0
                    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                else:
                    auc_score = precision = recall = f1 = 0
                
                results.append({
                    'k': k,
                    'contamination': contamination,
                    'auc_score': auc_score,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1,
                    'n_outliers_detected': np.sum(y_pred_binary == 1)
                })
                
            except Exception as e:
                print(f"Error with k={k}, contamination={contamination}: {e}")
                continue
    
    results_df = pd.DataFrame(results)
    
    # Find optimal parameters
    if len(results_df) > 0:
        best_idx = results_df['f1_score'].idxmax()
        optimal_params = {
            'k': results_df.loc[best_idx, 'k'],
            'contamination': results_df.loc[best_idx, 'contamination'],
            'best_f1': results_df.loc[best_idx, 'f1_score']
        }
    else:
        optimal_params = {'k': 20, 'contamination': 0.1, 'best_f1': 0}
    
    return {
        'results_df': results_df,
        'optimal_params': optimal_params
    }

def plot_lof_analysis(X, y, lof_scores, title="LOF Analysis", figsize=(15, 10)):
    """
    Comprehensive visualization of LOF analysis
    
    Parameters:
    -----------
    X : array-like
        Input features (2D for visualization)
    y : array-like
        True labels
    lof_scores : array-like
        LOF scores
    title : str
        Plot title
    figsize : tuple
        Figure size
    """
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    
    # 1. Scatter plot with LOF scores
    scatter = axes[0, 0].scatter(X[:, 0], X[:, 1], c=lof_scores, 
                                cmap='viridis', alpha=0.7, s=50)
    axes[0, 0].set_title(f'{title} - LOF Scores')
    axes[0, 0].set_xlabel('Feature 1')
    axes[0, 0].set_ylabel('Feature 2')
    plt.colorbar(scatter, ax=axes[0, 0], label='LOF Score')
    
    # 2. True vs Predicted
    colors = ['blue' if label == 0 else 'red' for label in y]
    axes[0, 1].scatter(X[:, 0], X[:, 1], c=colors, alpha=0.7, s=50)
    axes[0, 1].set_title('True Labels (Blue=Normal, Red=Outlier)')
    axes[0, 1].set_xlabel('Feature 1')
    axes[0, 1].set_ylabel('Feature 2')
    
    # 3. LOF Score Distribution
    normal_scores = lof_scores[y == 0]
    outlier_scores = lof_scores[y == 1]
    
    axes[1, 0].hist(normal_scores, bins=20, alpha=0.7, label='Normal', color='blue')
    axes[1, 0].hist(outlier_scores, bins=20, alpha=0.7, label='Outlier', color='red')
    axes[1, 0].axvline(x=1.0, color='black', linestyle='--', label='LOF=1.0')
    axes[1, 0].set_xlabel('LOF Score')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('LOF Score Distribution')
    axes[1, 0].legend()
    
    # 4. ROC Curve-like plot (Precision-Recall)
    if len(np.unique(y)) > 1:
        precision, recall, thresholds = precision_recall_curve(y, lof_scores)
        axes[1, 1].plot(recall, precision, marker='o', markersize=3)
        axes[1, 1].set_xlabel('Recall')
        axes[1, 1].set_ylabel('Precision')
        axes[1, 1].set_title('Precision-Recall Curve')
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'No Outliers\\nDetected', ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Precision-Recall (N/A)')
    
    plt.tight_layout()
    return fig

def evaluate_lof_performance(y_true, lof_scores, contamination=0.1):
    """
    Evaluate LOF performance with multiple metrics
    
    Parameters:
    -----------
    y_true : array-like
        True binary labels (0=normal, 1=outlier)
    lof_scores : array-like
        LOF scores from the algorithm
    contamination : float
        Contamination parameter used
    
    Returns:
    --------
    metrics : dict
        Performance metrics
    """
    metrics = {}
    
    # Convert scores to binary predictions using threshold
    threshold = np.percentile(lof_scores, (1 - contamination) * 100)
    y_pred = (lof_scores > threshold).astype(int)
    
    # Basic metrics
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    metrics['true_positives'] = tp
    metrics['false_positives'] = fp
    metrics['true_negatives'] = tn
    metrics['false_negatives'] = fn
    
    # Calculated metrics
    metrics['precision'] = tp / (tp + fp) if (tp + fp) > 0 else 0
    metrics['recall'] = tp / (tp + fn) if (tp + fn) > 0 else 0
    metrics['f1_score'] = 2 * metrics['precision'] * metrics['recall'] / (metrics['precision'] + metrics['recall']) if (metrics['precision'] + metrics['recall']) > 0 else 0
    metrics['accuracy'] = (tp + tn) / (tp + fp + tn + fn)
    
    # AUC if possible
    if len(np.unique(y_true)) > 1:
        metrics['auc_score'] = roc_auc_score(y_true, lof_scores)
    else:
        metrics['auc_score'] = 0
    
    # LOF-specific metrics
    metrics['avg_lof_normal'] = np.mean(lof_scores[y_true == 0])
    metrics['avg_lof_outlier'] = np.mean(lof_scores[y_true == 1]) if np.sum(y_true == 1) > 0 else 0
    metrics['max_lof_score'] = np.max(lof_scores)
    metrics['threshold_used'] = threshold
    
    return metrics

In [4]:
# Reusable LOF Pipeline Class
class LOFPipeline:
    """
    A comprehensive Local Outlier Factor (LOF) pipeline for anomaly detection
    
    This class provides a complete workflow for LOF analysis including:
    - Data preprocessing and scaling
    - Parameter optimization
    - Anomaly detection with LOF
    - Performance evaluation
    - Visualization tools
    """
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.scaler = StandardScaler()
        self.lof_model = None
        self.optimal_params = None
        self.lof_scores = None
        self.anomaly_labels = None
        self.evaluation_metrics = None
        self.is_fitted = False
        
    def preprocess_data(self, X, scaling_method='standard'):
        """
        Preprocess data for LOF analysis
        
        Parameters:
        -----------
        X : array-like
            Input features
        scaling_method : str
            'standard', 'minmax', or 'none'
        
        Returns:
        --------
        X_processed : array-like
            Preprocessed data
        """
        X_processed = np.array(X)
        
        if scaling_method == 'standard':
            self.scaler = StandardScaler()
            X_processed = self.scaler.fit_transform(X_processed)
            print("✅ Data standardized using StandardScaler")
        elif scaling_method == 'minmax':
            self.scaler = MinMaxScaler()
            X_processed = self.scaler.fit_transform(X_processed)
            print("✅ Data normalized using MinMaxScaler")
        elif scaling_method == 'none':
            print("⚠️ No scaling applied")
        
        print(f"📊 Processed data shape: {X_processed.shape}")
        return X_processed
    
    def optimize_parameters(self, X, y=None, k_range=None, contamination_range=None, metric='f1_score', cv_folds=3):
        """
        Optimize LOF parameters using grid search
        
        Parameters:
        -----------
        X : array-like
            Input features
        y : array-like, optional
            True labels for supervised optimization
        k_range : iterable
            Range of n_neighbors values to test
        contamination_range : iterable
            Range of contamination values to test
        metric : str
            Optimization metric ('f1_score', 'auc_score', 'precision', 'recall')
        cv_folds : int
            Number of cross-validation folds
        
        Returns:
        --------
        optimal_params : dict
            Best parameters found
        """
        if k_range is None:
            k_range = range(5, min(51, len(X)//2), 5)
        
        if contamination_range is None:
            # If we have true labels, estimate contamination
            if y is not None:
                true_contamination = np.sum(y == 1) / len(y)
                base_contamination = max(0.05, min(0.3, true_contamination))
                contamination_range = [base_contamination * 0.5, base_contamination, 
                                     base_contamination * 1.5, base_contamination * 2]
            else:
                contamination_range = [0.05, 0.1, 0.15, 0.2]
        
        print(f"🔍 Optimizing LOF parameters...")
        print(f"   - k_range: {list(k_range)}")
        print(f"   - contamination_range: {contamination_range}")
        print(f"   - optimization metric: {metric}")
        
        best_score = -np.inf
        best_params = {'n_neighbors': 20, 'contamination': 0.1}
        
        # If we have true labels, do supervised optimization
        if y is not None:
            analysis_results = analyze_lof_sensitivity(X, y, k_range, contamination_range)
            results_df = analysis_results['results_df']
            
            if len(results_df) > 0 and metric in results_df.columns:
                best_idx = results_df[metric].idxmax()
                best_params = {
                    'n_neighbors': int(results_df.loc[best_idx, 'k']),
                    'contamination': results_df.loc[best_idx, 'contamination']
                }
                best_score = results_df.loc[best_idx, metric]
                
                print(f"✅ Optimization complete!")
                print(f"   - Best {metric}: {best_score:.4f}")
                print(f"   - Best n_neighbors: {best_params['n_neighbors']}")
                print(f"   - Best contamination: {best_params['contamination']}")
            else:
                print("⚠️ Optimization failed, using default parameters")
        else:
            # Unsupervised optimization using internal metrics
            print("🔧 Unsupervised optimization using silhouette-based scoring...")
            for k in k_range:
                for contamination in contamination_range:
                    try:
                        lof = LocalOutlierFactor(n_neighbors=k, contamination=contamination)
                        y_pred = lof.fit_predict(X)
                        
                        # Use negative outlier factor as score
                        scores = -lof.negative_outlier_factor_
                        
                        # Simple scoring: prefer parameters that give reasonable outlier counts
                        n_outliers = np.sum(y_pred == -1)
                        expected_outliers = len(X) * contamination
                        outlier_ratio_score = 1 - abs(n_outliers - expected_outliers) / expected_outliers
                        
                        # Score based on LOF score distribution
                        score_std = np.std(scores)
                        combined_score = outlier_ratio_score * score_std
                        
                        if combined_score > best_score:
                            best_score = combined_score
                            best_params = {'n_neighbors': k, 'contamination': contamination}
                            
                    except Exception as e:
                        continue
            
            print(f"✅ Unsupervised optimization complete!")
            print(f"   - Best combined score: {best_score:.4f}")
        
        self.optimal_params = best_params
        return best_params
    
    def fit_predict(self, X, n_neighbors=None, contamination=None, use_optimized_params=True):
        """
        Fit LOF model and predict anomalies
        
        Parameters:
        -----------
        X : array-like
            Input features
        n_neighbors : int, optional
            Number of neighbors (uses optimized if available)
        contamination : float, optional
            Expected proportion of outliers
        use_optimized_params : bool
            Whether to use optimized parameters
        
        Returns:
        --------
        anomaly_labels : array-like
            Binary labels (-1 for outliers, 1 for inliers)
        lof_scores : array-like
            Local outlier factor scores
        """
        # Set parameters
        if use_optimized_params and self.optimal_params:
            params = self.optimal_params.copy()
        else:
            params = {
                'n_neighbors': n_neighbors or 20,
                'contamination': contamination or 0.1
            }
        
        # Override with explicitly provided parameters
        if n_neighbors is not None:
            params['n_neighbors'] = n_neighbors
        if contamination is not None:
            params['contamination'] = contamination
        
        print(f"🔧 Fitting LOF with parameters: {params}")
        
        # Fit LOF model
        self.lof_model = LocalOutlierFactor(**params)
        self.anomaly_labels = self.lof_model.fit_predict(X)
        self.lof_scores = -self.lof_model.negative_outlier_factor_
        self.is_fitted = True
        
        # Print summary
        n_outliers = np.sum(self.anomaly_labels == -1)
        outlier_percentage = (n_outliers / len(X)) * 100
        
        print(f"✅ LOF analysis complete!")
        print(f"   - Total samples: {len(X)}")
        print(f"   - Detected outliers: {n_outliers} ({outlier_percentage:.1f}%)")
        print(f"   - LOF score range: {self.lof_scores.min():.3f} to {self.lof_scores.max():.3f}")
        print(f"   - Mean LOF score: {self.lof_scores.mean():.3f}")
        
        return self.anomaly_labels, self.lof_scores
    
    def evaluate_performance(self, y_true):
        """
        Evaluate LOF performance against true labels
        
        Parameters:
        -----------
        y_true : array-like
            True binary labels (0=normal, 1=outlier)
        
        Returns:
        --------
        metrics : dict
            Performance metrics
        """
        if not self.is_fitted:
            raise ValueError("Model not fitted. Run fit_predict first.")
        
        # Convert LOF labels (-1, 1) to binary (1, 0)
        y_pred_binary = (self.anomaly_labels == -1).astype(int)
        
        # Get contamination parameter for threshold calculation
        contamination = self.lof_model.contamination
        
        self.evaluation_metrics = evaluate_lof_performance(y_true, self.lof_scores, contamination)
        
        return self.evaluation_metrics
    
    def plot_results(self, X, y_true=None, figsize=(16, 12), title="LOF Analysis Results"):
        """
        Plot comprehensive LOF analysis results
        
        Parameters:
        -----------
        X : array-like
            Original features (first 2 dimensions used for plotting)
        y_true : array-like, optional
            True labels for comparison
        figsize : tuple
            Figure size
        title : str
            Plot title
        
        Returns:
        --------
        fig : matplotlib.figure.Figure
            The figure object
        """
        if not self.is_fitted:
            raise ValueError("Model not fitted. Run fit_predict first.")
        
        # Use first 2 dimensions for visualization
        X_plot = X[:, :2] if X.shape[1] >= 2 else X
        
        if y_true is not None:
            return plot_lof_analysis(X_plot, y_true, self.lof_scores, title, figsize)
        else:
            # Plot without true labels
            fig, axes = plt.subplots(2, 2, figsize=figsize)
            
            # 1. LOF scores
            scatter = axes[0, 0].scatter(X_plot[:, 0], X_plot[:, 1], 
                                        c=self.lof_scores, cmap='viridis', alpha=0.7, s=50)
            axes[0, 0].set_title('LOF Scores')
            axes[0, 0].set_xlabel('Feature 1')
            axes[0, 0].set_ylabel('Feature 2')
            plt.colorbar(scatter, ax=axes[0, 0])
            
            # 2. Predicted outliers
            colors = ['blue' if label == 1 else 'red' for label in self.anomaly_labels]
            axes[0, 1].scatter(X_plot[:, 0], X_plot[:, 1], c=colors, alpha=0.7, s=50)
            axes[0, 1].set_title('Predicted Labels (Blue=Normal, Red=Outlier)')
            axes[0, 1].set_xlabel('Feature 1')
            axes[0, 1].set_ylabel('Feature 2')
            
            # 3. LOF score distribution
            normal_scores = self.lof_scores[self.anomaly_labels == 1]
            outlier_scores = self.lof_scores[self.anomaly_labels == -1]
            
            axes[1, 0].hist(normal_scores, bins=20, alpha=0.7, label='Predicted Normal', color='blue')
            if len(outlier_scores) > 0:
                axes[1, 0].hist(outlier_scores, bins=20, alpha=0.7, label='Predicted Outlier', color='red')
            axes[1, 0].axvline(x=1.0, color='black', linestyle='--', label='LOF=1.0')
            axes[1, 0].set_xlabel('LOF Score')
            axes[1, 0].set_ylabel('Frequency')
            axes[1, 0].set_title('LOF Score Distribution')
            axes[1, 0].legend()
            
            # 4. Summary statistics
            stats_text = f"""LOF Analysis Summary:
            
Total Samples: {len(X)}
Detected Outliers: {np.sum(self.anomaly_labels == -1)}
Outlier Percentage: {(np.sum(self.anomaly_labels == -1) / len(X)) * 100:.1f}%

LOF Score Statistics:
Mean: {self.lof_scores.mean():.3f}
Std: {self.lof_scores.std():.3f}
Min: {self.lof_scores.min():.3f}
Max: {self.lof_scores.max():.3f}

Parameters Used:
n_neighbors: {self.lof_model.n_neighbors}
contamination: {self.lof_model.contamination}"""
            
            axes[1, 1].text(0.05, 0.95, stats_text, transform=axes[1, 1].transAxes,verticalalignment='top', fontsize=10,bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
            axes[1, 1].set_xlim(0, 1)
            axes[1, 1].set_ylim(0, 1)
            axes[1, 1].axis('off')
            axes[1, 1].set_title('Analysis Summary')
            
            plt.tight_layout()
            return fig
    
    def predict_new_data(self, X_new):
        """
        Predict anomalies for new data points
        
        Parameters:
        -----------
        X_new : array-like
            New data points
        
        Returns:
        --------
        anomaly_scores : array-like
            Anomaly scores for new points
        """
        if not self.is_fitted:
            raise ValueError("Model not fitted. Run fit_predict first.")
        
        # Scale new data using fitted scaler
        if hasattr(self.scaler, 'transform'):
            X_new_scaled = self.scaler.transform(X_new)
        else:
            X_new_scaled = X_new
        
        # Note: LOF doesn't directly support new data prediction
        # This is a limitation of the algorithm
        print("⚠️ Warning: LOF doesn't support direct prediction on new data.")
        print("   Consider retraining with combined data or using other methods.")
        
        return None
    
    def save_results(self, filepath, X_original=None):
        """
        Save LOF analysis results
        
        Parameters:
        -----------
        filepath : str
            Path to save results
        X_original : array-like, optional
            Original data to include
        """
        if not self.is_fitted:
            raise ValueError("No results to save. Run fit_predict first.")
        
        results = {
            'anomaly_labels': self.anomaly_labels,
            'lof_scores': self.lof_scores,
            'optimal_params': self.optimal_params or {},
            'evaluation_metrics': self.evaluation_metrics or {}
        }
        
        if X_original is not None:
            results['original_data'] = X_original
        
        np.savez(filepath, **results)
        print(f"💾 Results saved to {filepath}")

# Practical Demonstration
def demonstrate_lof_analysis():
    """
    Comprehensive demonstration of LOF analysis
    """
    print("🚀 Starting comprehensive LOF demonstration...")
    print("="*60)
    
    # Create datasets
    datasets = create_anomaly_datasets()
    
    pipeline = LOFPipeline(random_state=42)
    
    for name, (X, y) in datasets.items():
        print(f"\\n{'='*60}")
        print(f"📊 ANALYZING DATASET: {name.upper()}")
        print(f"{'='*60}")
        print(f"Data shape: {X.shape}")
        print(f"True outliers: {np.sum(y == 1)} ({(np.sum(y == 1)/len(y)*100):.1f}%)")
        
        # Preprocess data
        X_processed = pipeline.preprocess_data(X, scaling_method='standard')
        
        # Optimize parameters
        print("\\n🔍 Optimizing parameters...")
        optimal_params = pipeline.optimize_parameters(X_processed, y)
        
        # Fit and predict
        print("\\n🔧 Running LOF analysis...")
        anomaly_labels, lof_scores = pipeline.fit_predict(X_processed, use_optimized_params=True)
        
        # Evaluate performance
        print("\\n📊 Evaluating performance...")
        metrics = pipeline.evaluate_performance(y)
        
        print("\\nPerformance Metrics:")
        print("-" * 30)
        for metric_name, value in metrics.items():
            if isinstance(value, (int, float)):
                print(f"{metric_name.replace('_', ' ').title():<25}: {value:.4f}")
        
        # Plot results
        print("\\n📈 Generating visualizations...")
        pipeline.plot_results(X_processed, y, title=f'LOF Analysis: {name}')
        plt.show()
        
        print(f"\\n✅ Analysis of {name} dataset complete!")
    
    print("\\n🎉 All LOF demonstrations completed successfully!")
    
    return pipeline